# Nonlinear minimization problems (Unit 3.8)

We consider problems of the form
$$\text{find } u \in V \text{ s.t. } E(u) \leq E(v)\ \ \forall  v \in V.$$

In [1]:
import netgen.gui
%gui tk
import tkinter
from ngsolve import *

## Scalar minimization problems

As a first example we take $V=H_0^1$ and
$$E(u)=\int_\Omega |\nabla u|^2 + u^4−fu dx.$$
The minimization is equivalent to solving the nonlinear PDE:
$$−\Delta u+4u^3=f \text{ in }\Omega$$
We solve the PDE with a Newton iteration.

In [2]:
from netgen.geom2d import unit_square

mesh = Mesh (unit_square.GenerateMesh(maxh=0.2))
V = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = V.TrialFunction()


To solve the problem we use the `SymbolicEnergy` integrator. Based on the symbolic description of the energy functional, it is able to 
* evaluate the energy functional (`Energy`)
$$E(u) \qquad (E:V \to \mathbb{R})$$
* compute the Gateau derivative for a given $u$ (`Apply`):
$$A(u)(v) = E'(u)(v) \qquad (A(u): V \to \mathbb{R})$$
* compute the second derivative (`AssembleLinearization`)
$$(\delta A)(w)(u,v) \qquad (\delta A(w): V\times V \to \mathbb{R})$$

In [3]:
a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy ( grad(u)*grad(u) + u**4-u )


Equivalent to:
```
a += SymbolicBFI( 2 * grad(u) * grad(v) + 4*u*u*u*v - 1 * v)
``` 
(which has the same form as the problems in [the nonlinear example](../unit-3.5-nonlinear/nonlinear.ipynb))

We recall the Newton iteration (cf. unit-3.5 ) we make the loop:
- Given an initial guess $u_0$ 
- loop over $i=0,\dots$ until convergence:

   - Compute linearization: $A u^i+\delta A(u^i)\Delta u^i=0$:

      - $f^i=Au^i$

      - $B^i=\delta A(u^i)$
      - Solve $B^i\Delta u^i=−f^i$

   - Update $u^{i+1}=u^i+\Delta u^i$

   - Evaluate stopping criteria

- Evaluate $E(u^{i+1})$

As a stopping criteria we take $\langle Au^i,\Delta u^i\rangle=\langle Au^i,Au^i\rangle_{(B^i)^{−1}}<\epsilon$
.

In [4]:
def SolveNonlinearMinProblem(a,gfu,tol=1e-13,maxits=25):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()

    for it in range(maxits):
        print ("Newton iteration {:3}".format(it),end="")
        print ("energy = {:16}".format(a.Energy(gfu.vec)),end="")
    
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        inv = a.mat.Inverse(V.FreeDofs())
        du.data = inv * res
    
        #update iteration
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break
        Redraw(blocking=True)

In [5]:
gfu = GridFunction (V)
gfu.vec[:] = 0
Draw(gfu,mesh,"u")

SolveNonlinearMinProblem(a,gfu)

print ("energy = ", a.Energy(gfu.vec))
    

Newton iteration   0energy =              0.0<A u 0 , A u 0 >_{-1}^0.5 =  0.13255958157127926
Newton iteration   1energy = -0.008785677986915064<A u 1 , A u 1 >_{-1}^0.5 =  1.1107597333570957e-05
Newton iteration   2energy = -0.008785678048604426<A u 2 , A u 2 >_{-1}^0.5 =  2.807448411922022e-13
Newton iteration   3energy = -0.008785678048604428<A u 3 , A u 3 >_{-1}^0.5 =  3.4543927308271275e-17
energy =  -0.008785678048604428


## Nonlinear elasticity

We consider a beam which is fixed on one side and is subject to gravity only. We assume a Neo-Hookean hyperelastic material. The model is a nonlinear minimization problem with
$$E(v):=\int_\Omega \mu^2 (\text{tr}(F^T F - I) + \frac{2 \mu}{\lambda} \text{det}(F^TF)^{\frac{\lambda}{ 2 \mu}} - 1) - \gamma (f,v)\ dx$$
where $\mu$ and $\lambda$ are the Lamé parameters and $F=I+Du$ where $v:\Omega \rightarrow \mathbb{R}^2$ is the sought for displacement.

In [6]:
import netgen.geom2d as geom2d
from ngsolve import *

geo = geom2d.SplineGeometry()
pnums = [ geo.AddPoint (x,y,maxh=0.01) for x,y in [(0,0), (1,0), (1,0.1), (0,0.1)] ]
for p1,p2,bc in [(0,1,"bot"), (1,2,"right"), (2,3,"top"), (3,0,"left")]:
     geo.Append(["line", pnums[p1], pnums[p2]], bc=bc)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

In [7]:
# E module and poisson number:
E, nu = 210, 0.2
# Lamé constants:
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

V = H1(mesh, order=2, dirichlet="left", dim=mesh.dim)
u  = V.TrialFunction()

#gravity:
force = CoefficientFunction( (0,-1) )


In [8]:

# some utils:
# These are no longer needed -- provided now
# def IdentityCF(dim):
#     return CoefficientFunction( tuple( [1 if i==j else 0 for i in range(dim) for j in range(dim)]), dims=(dim,dim) )
# this is 'Id' now

# def Trace(mat):
#     return sum( [mat[i,i] for i in range(mat.dims[0]) ])

# def Det(mat):
#     if mat.dims[0] == 2:
#         return mat[0,0]*mat[1,1]-mat[0,1]*mat[1,0]

def Pow(a, b):
    return exp (log(a)*b)
    
def NeoHook (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C), -lam/2/mu) - 1)

I = Id(mesh.dim)
F = I + u.Deriv()   # attention: row .. component, col .. derivative
C = F * F.trans  

factor = Parameter(1.0)

a = BilinearForm(V, symmetric=False)
a += SymbolicEnergy(  NeoHook (C).Compile() )
a += SymbolicEnergy(  (-factor * InnerProduct(force,u) ).Compile() )

We want to solve the minimization problem for $\gamma=5$. Due to the high nonlinearity in the problem, the Newton iteration will not convergence with any initial guess. We approach the case $\gamma=5$ by solving problems with $\gamma=i/10$ for $i=1,\dots,50$ and taking the solution of the previous problem as an initial guess.

In [9]:
gfu = GridFunction(V)
gfu.vec[:] = 0

Draw (gfu, mesh, "u")
SetVisualization (deformation=True)

res = gfu.vec.CreateVector()
du = gfu.vec.CreateVector()

for loadstep in range(50):
    print ("loadstep", loadstep)
    factor.Set ((loadstep+1)/10)
    SolveNonlinearMinProblem(a,gfu)
    Redraw()

loadstep 0
Newton iteration   0energy = 8.749999999999996<A u 0 , A u 0 >_{-1}^0.5 =  0.016678457455895263
Newton iteration   1energy = 8.750132589738278<A u 1 , A u 1 >_{-1}^0.5 =  0.023333456941189643
Newton iteration   2energy = 8.74986114964883<A u 2 , A u 2 >_{-1}^0.5 =  0.00010232478378596959
Newton iteration   3energy = 8.749861144413645<A u 3 , A u 3 >_{-1}^0.5 =  5.165622071333366e-08
Newton iteration   4energy = 8.749861144413641<A u 4 , A u 4 >_{-1}^0.5 =  2.752957086425484e-13
Newton iteration   5energy = 8.749861144413641<A u 5 , A u 5 >_{-1}^0.5 =  9.862114249553194e-16
loadstep 1
Newton iteration   0energy = 8.74958389081265<A u 0 , A u 0 >_{-1}^0.5 =  0.016596121926300306
Newton iteration   1energy = 8.749710095594017<A u 1 , A u 1 >_{-1}^0.5 =  0.022958360130497214
Newton iteration   2energy = 8.749447298577934<A u 2 , A u 2 >_{-1}^0.5 =  0.00014216303112616537
Newton iteration   3energy = 8.749447288578642<A u 3 , A u 3 >_{-1}^0.5 =  2.325777629349615e-06
Newton itera

Newton iteration   5energy = 8.727969632143362<A u 5 , A u 5 >_{-1}^0.5 =  8.81177880268392e-15
loadstep 14
Newton iteration   0energy = 8.725297521130479<A u 0 , A u 0 >_{-1}^0.5 =  0.009929570974360123
Newton iteration   1energy = 8.725258453148317<A u 1 , A u 1 >_{-1}^0.5 =  0.004198069524844316
Newton iteration   2energy = 8.72524963531322<A u 2 , A u 2 >_{-1}^0.5 =  9.913698495326403e-05
Newton iteration   3energy = 8.725249630399578<A u 3 , A u 3 >_{-1}^0.5 =  6.17699982014573e-07
Newton iteration   4energy = 8.72524963039939<A u 4 , A u 4 >_{-1}^0.5 =  2.705157522038129e-13
Newton iteration   5energy = 8.725249630399393<A u 5 , A u 5 >_{-1}^0.5 =  7.519895571236739e-15
loadstep 15
Newton iteration   0energy = 8.722483105652495<A u 0 , A u 0 >_{-1}^0.5 =  0.009507271723498576
Newton iteration   1energy = 8.722445739330311<A u 1 , A u 1 >_{-1}^0.5 =  0.0036233383772834213
Newton iteration   2energy = 8.72243916956892<A u 2 , A u 2 >_{-1}^0.5 =  8.084104221351204e-05
Newton iterati

Newton iteration   3energy = 8.672992297826848<A u 3 , A u 3 >_{-1}^0.5 =  5.648738194817864e-10
Newton iteration   4energy = 8.672992297826848<A u 4 , A u 4 >_{-1}^0.5 =  9.693425332525348e-15
loadstep 31
Newton iteration   0energy = 8.669376547136705<A u 0 , A u 0 >_{-1}^0.5 =  0.005586260499043558
Newton iteration   1energy = 8.669361347518183<A u 1 , A u 1 >_{-1}^0.5 =  0.0005444414158932738
Newton iteration   2energy = 8.669361199168447<A u 2 , A u 2 >_{-1}^0.5 =  3.1366272721981807e-06
Newton iteration   3energy = 8.669361199163518<A u 3 , A u 3 >_{-1}^0.5 =  3.8234160492094605e-10
Newton iteration   4energy = 8.669361199163525<A u 4 , A u 4 >_{-1}^0.5 =  1.0157142608686646e-14
loadstep 32
Newton iteration   0energy = 8.665715002539416<A u 0 , A u 0 >_{-1}^0.5 =  0.005450322534479756
Newton iteration   1energy = 8.66570050797302<A u 1 , A u 1 >_{-1}^0.5 =  0.0004961380054855532
Newton iteration   2energy = 8.665700384783833<A u 2 , A u 2 >_{-1}^0.5 =  2.6203482141725422e-06
Newto

Newton iteration   2energy = 8.607964638432573<A u 2 , A u 2 >_{-1}^0.5 =  2.538102250213158e-07
Newton iteration   3energy = 8.607964638432543<A u 3 , A u 3 >_{-1}^0.5 =  1.7115838604748933e-12
Newton iteration   4energy = 8.607964638432543<A u 4 , A u 4 >_{-1}^0.5 =  1.1995707999536518e-14
loadstep 48
Newton iteration   0energy = 8.603971674969976<A u 0 , A u 0 >_{-1}^0.5 =  0.004009820942033808
Newton iteration   1energy = 8.603963727064613<A u 1 , A u 1 >_{-1}^0.5 =  0.00015111333105232534
Newton iteration   2energy = 8.603963715642134<A u 2 , A u 2 >_{-1}^0.5 =  2.220011026250392e-07
Newton iteration   3energy = 8.603963715642113<A u 3 , A u 3 >_{-1}^0.5 =  1.275882735630376e-12
Newton iteration   4energy = 8.603963715642115<A u 4 , A u 4 >_{-1}^0.5 =  1.0757110515476966e-14
loadstep 49
Newton iteration   0energy = 8.599954912409892<A u 0 , A u 0 >_{-1}^0.5 =  0.003950356154692413
Newton iteration   1energy = 8.599947195556835<A u 1 , A u 1 >_{-1}^0.5 =  0.00014248841787251092
New

## Allen-Cahn equation

The Allen-Cahn equations describe the process of phase separation and is the ($L^2$) gradient-flow equation to the energy
$$E(v)=\int_\Omega \epsilon |\nabla v|^2 + v^2(1-v^2) dx$$

i.e. the solution to the Allen-Cahn equation solves
$$\partial_t u=\frac{\delta E}{ \delta u}$$
The quantity $u$ is an indicator for a phase where $−1$ refers to one phase and $1$ to another phase. 

The equation has two driving forces:

- $u$ is pulled into one of the two minima $(−1 \text{ and } 1)$ of the nonlinear term $u^2(1−u^2)$ (separation of the phases)
- the diffusion term scaled with \epsilon enforces a smooth transition between the two phases.  $\epsilon$ determines the size of the transition layer

We use the "SymbolicEnergy" feature to formulate the energy minimization problem and combine it with an implicit Euler discretization:
$$Mu^{n+1}−Mu^n=\Delta t\underbrace{\frac{\delta E}{\delta u}}_{=:A(u)} (u^{n+1})$$
which we can interpret as a nonlinear minimization problem again with the energy
$$E^{IE}(v)=\int_\Omega \frac{\varepsilon}{2} \vert \nabla v \vert^2~+~v^2(1-v^2) + \frac{1}{2 \Delta t} |v−u^n|^2 dx$$

To solve the nonlinear equation at every time step we again rely on Newton's method.

In [10]:
from ngsolve import *

from netgen.geom2d import *


periodic = SplineGeometry()
pnts = [ (0,0), (1,0), (1,1), (0,1) ]
pnums = [periodic.AppendPoint(*p) for p in pnts]

lright = periodic.Append ( ["line", pnums[0], pnums[1]],bc="periodic")
btop = periodic.Append ( ["line", pnums[1], pnums[2]], bc="periodic")
periodic.Append ( ["line", pnums[3], pnums[2]], leftdomain=0, rightdomain=1, copy=lright, bc="periodic")
periodic.Append ( ["line", pnums[0], pnums[3]], leftdomain=0, rightdomain=1, copy=btop, bc="periodic")

mesh = Mesh (periodic.GenerateMesh(maxh=0.2))

V = Periodic(H1(mesh, order=4, dirichlet=[]))

u = V.TrialFunction()

eps = 4e-3
dt = 1e-1

gfu = GridFunction(V)
gfuold = GridFunction(V)

a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy (eps/2*grad(u)*grad(u)
                     + ((1-u*u)*(1-u*u))
                     + 0.5/dt*(u-gfuold)*(u-gfuold))

In [11]:
from math import pi
gfu = GridFunction(V)
gfu.Set(sin(2*pi*x))
#gfu.Set(sin(1e8*x)) #<- essentially a random function
Draw(gfu,mesh,"u")
SetVisualization (deformation=False)
t = 0

In [12]:
for timestep in range(50):
    gfuold.vec.data = gfu.vec
    SolveNonlinearMinProblem(a,gfu)
    Redraw() 
    t += dt
    print("t = ", t)

Newton iteration   0energy = 0.41450268691109693<A u 0 , A u 0 >_{-1}^0.5 =  0.2809232585283062
Newton iteration   1energy = 0.3769002953873679<A u 1 , A u 1 >_{-1}^0.5 =  0.01917619265956576
Newton iteration   2energy = 0.3767158234035436<A u 2 , A u 2 >_{-1}^0.5 =  9.63037928498279e-05
Newton iteration   3energy = 0.3767158187662532<A u 3 , A u 3 >_{-1}^0.5 =  2.5083857940526133e-09
Newton iteration   4energy = 0.37671581876625293<A u 4 , A u 4 >_{-1}^0.5 =  3.019981494439342e-16
t =  0.1
Newton iteration   0energy = 0.3438885235450326<A u 0 , A u 0 >_{-1}^0.5 =  0.23599828005106366
Newton iteration   1energy = 0.3171273710024067<A u 1 , A u 1 >_{-1}^0.5 =  0.01345215122130782
Newton iteration   2energy = 0.31703668361020804<A u 2 , A u 2 >_{-1}^0.5 =  4.664802779278362e-05
Newton iteration   3energy = 0.3170366825221797<A u 3 , A u 3 >_{-1}^0.5 =  5.774609387010979e-10
Newton iteration   4energy = 0.31703668252217976<A u 4 , A u 4 >_{-1}^0.5 =  3.2305269197519717e-16
t =  0.2
Newton

Newton iteration   1energy = 0.2398221015429813<A u 1 , A u 1 >_{-1}^0.5 =  1.7245245368126424e-06
Newton iteration   2energy = 0.23982210154149428<A u 2 , A u 2 >_{-1}^0.5 =  1.897987146648432e-12
Newton iteration   3energy = 0.23982210154149422<A u 3 , A u 3 >_{-1}^0.5 =  4.2653240360681535e-16
t =  2.1000000000000005
Newton iteration   0energy = 0.23982057039299465<A u 0 , A u 0 >_{-1}^0.5 =  0.0017251940955923981
Newton iteration   1energy = 0.23981908247720607<A u 1 , A u 1 >_{-1}^0.5 =  1.6048109259709833e-06
Newton iteration   2energy = 0.23981908247591832<A u 2 , A u 2 >_{-1}^0.5 =  1.6127005812911285e-12
Newton iteration   3energy = 0.23981908247591832<A u 3 , A u 3 >_{-1}^0.5 =  4.2841381204830243e-16
t =  2.2000000000000006
Newton iteration   0energy = 0.23981763600544376<A u 0 , A u 0 >_{-1}^0.5 =  0.00167729240859014
Newton iteration   1energy = 0.23981622954736212<A u 1 , A u 1 >_{-1}^0.5 =  1.4963465114914386e-06
Newton iteration   2energy = 0.23981622954624257<A u 2 , A

Newton iteration   1energy = 0.23978260925386752<A u 1 , A u 1 >_{-1}^0.5 =  4.858297225154298e-07
Newton iteration   2energy = 0.23978260925374953<A u 2 , A u 2 >_{-1}^0.5 =  1.2047592869405793e-13
Newton iteration   3energy = 0.2397826092537495<A u 3 , A u 3 >_{-1}^0.5 =  4.0494173366938454e-16
t =  4.200000000000001
Newton iteration   0energy = 0.23978207156317305<A u 0 , A u 0 >_{-1}^0.5 =  0.0010257547717514445
Newton iteration   1energy = 0.2397815454945841<A u 1 , A u 1 >_{-1}^0.5 =  4.617412570049385e-07
Newton iteration   2energy = 0.2397815454944776<A u 2 , A u 2 >_{-1}^0.5 =  1.080437194070748e-13
Newton iteration   3energy = 0.23978154549447764<A u 3 , A u 3 >_{-1}^0.5 =  4.0884591245457357e-16
t =  4.300000000000001
Newton iteration   0energy = 0.23978103071686074<A u 0 , A u 0 >_{-1}^0.5 =  0.0010037991768567138
Newton iteration   1energy = 0.23978052692656404<A u 1 , A u 1 >_{-1}^0.5 =  4.3918936193175685e-07
Newton iteration   2energy = 0.23978052692646756<A u 2 , A u 2